# A* Algorithm Implmentation: Closest Store by BART

**imports**

In [16]:
import neo4j
import math
import numpy as np
import pandas as pd
import re
import random


from scipy import spatial
from geographiclib.geodesic import Geodesic

**Driver Connection**

In [2]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

**Helper Functions**

In [43]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def my_neo4j_create_customer_node(customer_id, zip_code):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Customer {
                        customer_id: $customer_id,
                        zip_code: $zip_code
                    })
    
    """
    
    session.run(query, customer_id=customer_id, zip_code = zip_code)

def create_relationship_one_way_customer_station(from_customer, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Customer), 
          (to:Station)
    WHERE from.customer_id = $from_customer and to.name = $to_station
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_customer=from_customer, to_station=to_station, weight=weight)
    
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

def random_point(p, m): 
    l, r, t, b = my_calculate_box(p, m)
    
    random_latitude = random.uniform(l[0], r[0])
    random_longitude = random.uniform(b[1], t[1])
        
    return (random_latitude, random_longitude)

### Customer Zip Code -> Random (lat, long) pairs

In [44]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT customers.*, zip_codes.latitude, zip_codes.longitude
FROM customers
left join zip_codes 
on customers.zip = zip_codes.zip 

"""

customers = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customers.head()

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance,latitude,longitude
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1,37.8343,-122.2643
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1,37.8343,-122.2643
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1,37.8343,-122.2643
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1,37.8343,-122.2643
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1,37.8343,-122.2643


In [48]:
customers.apply(lambda x: random_point((x['latitude'], x['longitude']), 1), axis=1).value_counts()

(25.753499116426344, -80.2711)     419
(47.68819799762957, -122.3815)     315
(47.71639799566192, -122.2982)     296
(37.80409858265136, -122.207)      260
(37.921598576678136, -122.2984)    258
                                  ... 
(32.531398833332965, -96.6248)       1
(32.74199882390503, -96.4519)        1
(37.497598598144755, -122.27)        1
(32.72029882487848, -97.1565)        1
(37.71309858726455, -122.018)        1
Length: 550, dtype: int64

In [49]:
customers["random_loc"] =  customers.apply(lambda x: random_point((x['latitude'], x['longitude']), 1), axis=1)

In [50]:
customers.head()

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance,latitude,longitude,random_loc
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.83429858111791, -122.2643)"
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.83429858111791, -122.2643)"
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.83429858111791, -122.2643)"
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.83429858111791, -122.2643)"
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.83429858111791, -122.2643)"


In [51]:
 ## find the distance between each customer and all the bart stations 
